In [1]:
import pandas as pd

In [47]:
df = pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/data/2_reward_instances_4_80part1.csv', index_col=0)
df.head()

/Users/valarian/python/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,0,1,0,22,29,71,59,2,1,0,1,0,22,29,71,59,-5,[3]
1,1,0,0,1,0,22,29,70,59,2,0,0,1,0,22,29,70,59,-15,[3]
2,1,0,1,1,0,22,29,69,59,2,0,1,1,0,22,29,69,59,-15,[3]
3,1,0,0,1,0,22,29,68,59,0,0,0,1,0,22,29,68,59,-5,[3]
4,1,2,0,1,0,21,29,69,59,3,2,0,1,0,21,29,69,59,-15,[3]


In [138]:
df.groupby(by='0').count()['1'].min()

66

In [93]:
from collections import deque
import random

In [71]:
class Memory():
    
    def __init__(self,memsize):
        self.memsize = memsize
        self.memory = deque(maxlen=self.memsize)
    
    def add_episode(self,epsiode):
        self.memory.append(epsiode)
    
    def get_batch(self,bsize): #,time_step):
        sampled_epsiodes = random.sample(self.memory,bsize)
        batch = []
        for episode in sampled_epsiodes:
            #point = np.random.randint(0,len(episode)+1-time_step)
            batch.append(episode)
        return batch
mem = Memory(30)

In [67]:
df.drop(labels='19', axis=1, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,1,0,1,0,22,29,71,59,2,1,0,1,0,22,29,71,59,-5
1,1,0,0,1,0,22,29,70,59,2,0,0,1,0,22,29,70,59,-15
2,1,0,1,1,0,22,29,69,59,2,0,1,1,0,22,29,69,59,-15
3,1,0,0,1,0,22,29,68,59,0,0,0,1,0,22,29,68,59,-5
4,1,2,0,1,0,21,29,69,59,3,2,0,1,0,21,29,69,59,-15


In [86]:
df['0'].unique()

array([    1,     2,     3, ..., 49998, 49999, 50000])

In [75]:
sumr = df.groupby(by=['0']).sum()['18']
sumr[:5]

0
1   -1033
2   -1029
3   -1034
4   -1003
5   -1054
Name: 18, dtype: int64

In [ ]:
import torch
torch.from_numpy(df[df['0']==1].iloc[:,1:9].values)

In [ ]:
for i in range(30):
    bulk = df[df['0']==i+1].values
    s = bulk[:,1:9]
    a = bulk[:,9]
    r = bulk[:,18]
    s_ = bulk[:,10:18]
    mem.add_episode([s, a, r, s_])
mem.get_batch(1)[0]

In [105]:
torch.from_numpy(mem.get_batch(1)[0][0]).view(1,-1,8).size()

torch.Size([1, 70, 8])

In [109]:
torch.zeros(1,2,3).float().to('cpu').size()

torch.Size([1, 2, 3])

In [110]:
lstm = nn.LSTM(3, 3)

In [147]:
hidden = (torch.randn(1,2,3), torch.randn(1,2,3))

In [191]:
input1 = torch.randn(2,2,3)
output = lstm(input1, hidden)

In [192]:
re = output[0][:,1,:]
re

tensor([[-0.2221,  0.4182,  0.2360],
        [-0.1310,  0.3117,  0.2409]], grad_fn=<SliceBackward>)

In [195]:
re.max(dim=1)

(tensor([0.4182, 0.3117], grad_fn=<MaxBackward0>), tensor([1, 1]))

In [178]:
?torch.autograd.Variable.expand

Object `torch.autograd.Variable.expand` not found.


In [173]:
torch.max(output[0],2)

(tensor([[0.1931, 0.2727]], grad_fn=<MaxBackward0>), tensor([[1, 1]]))

In [18]:
df['18'].mean()

-12.854220791050528

In [9]:
?df.groupby

In [26]:
import torch
import torch.nn as nn

In [38]:
class DRQN(nn.Module):
    def __init__(self):
        super(DRQN, self).__init__()
        self.lstm = nn.LSTM(input_size=16, hidden_size=20, num_layers=1)
        self.fc1 = nn.Linear(20, 8)
        
    def forward(self, x):
        self.hidden_state = torch.randn(1, 1, 20)
        x, self.hidden_state = self.lstm(x, (self.hidden_state, self.hidden_state))
        x = self.fc1(x)
        return x
    
net = DRQN()
y = torch.randn(2, 1, 16).view(2,1,-1)
print(y)

net(y)

tensor([[[-0.6658, -0.0584,  1.9272,  0.1034,  0.5971,  0.5768, -0.4036,
          -0.6080, -0.1948,  0.5293, -1.5935,  0.0767,  0.7897, -1.2279,
          -0.4383, -1.4026]],

        [[-0.1709,  0.1526, -2.5830,  1.5564,  1.3877, -0.6615, -0.8710,
          -1.1732, -0.0234, -1.2946,  2.3732, -0.5207, -2.5881,  2.2847,
           0.7181,  0.6564]]])


tensor([[[-0.1694, -0.1055,  0.4178, -0.2490,  0.0935,  0.2321, -0.2570,
           0.2543]],

        [[-0.1088, -0.1865,  0.1173, -0.1290, -0.0600,  0.3244, -0.0115,
           0.2114]]], grad_fn=<AddBackward0>)

In [197]:
import gym
env = gym.make('CartPole-v0')
env.reset()

[2019-08-26 15:19:17,661] Making new env: CartPole-v0


array([-0.01243724, -0.01230862, -0.01885951,  0.03786528])

In [198]:
?torch.nn.Conv1d